# DQN with Ensembling

As a personal curiosity, I'd like to try to squeeze as much performance as possible from the initial DQN application.

I intend to do so using ensembling: I want to train multiple agents with the same video and different subjects and then use **majority-voting** at test time, to see if we can get any performance improvements, even with an extremely simple approach!

In [1]:
import tianshou as ts 
from tianshou.utils import TensorboardLogger

import torch
from torch import nn
from torch.utils.tensorboard import SummaryWriter

import numpy as np

import os
from datetime import datetime

In [2]:
from utils_preprocess import compute_frame_features, compute_foa_features

from env_base import BaseEnvironment

/Users/johndoe/Desktop/uni/affectivecompute/project/lib/python3.11/site-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists


## Data and environment initialisation

In [3]:
vid_filename = "001"
mat_filename = vid_filename + ".mat"

n_subjects = 10 # pick the first ten subjects (out of 39)
subjects = [n for n in range(n_subjects)]

In [4]:
patch_bounding_boxes_per_frame, patch_centres_per_frame, speaker_info_per_frame = compute_frame_features(
    vid_filename
)

foa_centres_per_frame_per_subject, patch_weights_per_frame = compute_foa_features(
    mat_filename, patch_centres_per_frame
)

foa_centres_per_frame_subjects = [
    [frame[target_subject] for frame in foa_centres_per_frame_per_subject] 
    for target_subject in subjects
    ]

In [8]:
markov_envs = []
for subject in subjects:
    markov_env = BaseEnvironment(
        1,
        patch_bounding_boxes_per_frame,
        patch_centres_per_frame,
        speaker_info_per_frame,
        foa_centres_per_frame_subjects[subject],
        patch_weights_per_frame,
        frame_width=320, # from data_utils.py
        frame_height=180,
    )
    markov_envs.append(markov_env)

In [7]:
all_train_envs = []
all_test_envs = []
for markov_env in markov_envs:
    num_train_envs = 5
    num_test_envs = 10

    train_envs = ts.env.DummyVectorEnv([lambda: markov_env for _ in range(num_train_envs)])
    all_train_envs.append(train_envs)

    test_envs = ts.env.DummyVectorEnv([lambda: markov_env for _ in range(num_test_envs)])
    all_test_envs.append(test_envs)

## DQN

First, let's construct the network.

The biggest headache comes from the observations: they're quite complex. So, we build multiple networks, each processing a part of an observation and combining their outputs in the end!

In [8]:
class Net(nn.Module):
    def __init__(self, observation_space, action_shape):
        super().__init__()

        self.num_patches = observation_space['patch_centres'].shape[0]

        # network for patch_centres
        self.patch_centres_net = nn.Sequential(
            nn.Linear(np.prod(observation_space['patch_centres'].shape), 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True)
        )

        # network for patch_bounding_boxes
        self.patch_bboxes_net = nn.Sequential(
            nn.Linear(np.prod(observation_space['patch_bounding_boxes'].shape), 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 64),
            nn.ReLU(inplace=True)
        )

        # network for speaker_info
        self.speaker_info_net = nn.Sequential(
            nn.Linear(np.prod(observation_space['speaker_info'].shape), 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 32),
            nn.ReLU(inplace=True)
        )

        # combining the outputs of all networks
        self.combined_net = nn.Sequential(
            nn.Linear(64 + 64 + 32, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, 128),
            nn.ReLU(inplace=True),
            nn.Linear(128, np.prod(action_shape))
        )

    def forward(self, obs, state=None, info={}):
        patch_centres = torch.tensor(obs['patch_centres'], dtype=torch.float32)
        patch_bboxes = torch.tensor(obs['patch_bounding_boxes'], dtype=torch.float32)
        speaker_info = torch.tensor(obs['speaker_info'], dtype=torch.float32)

        patch_centres = patch_centres.view(patch_centres.size(0), -1)
        patch_bboxes = patch_bboxes.view(patch_bboxes.size(0), -1)
        speaker_info = speaker_info.view(speaker_info.size(0), -1)

        # pass through respective networks
        patch_centres_out = self.patch_centres_net(patch_centres)
        patch_bboxes_out = self.patch_bboxes_net(patch_bboxes)
        speaker_info_out = self.speaker_info_net(speaker_info)

        # combine outputs
        combined = torch.cat([patch_centres_out, patch_bboxes_out, speaker_info_out], dim=1)

        logits = self.combined_net(combined)

        return logits, state

In [9]:
nets = []
optims = []
for markov_env in markov_envs:
    state_shape = markov_env.observation_space
    action_shape = markov_env.action_space.n

    net = Net(state_shape, action_shape)
    nets.append(net)

    optim = torch.optim.Adam(net.parameters(), lr=1e-3)
    optims.append(optim)

## Setting up DQN

First, we need to set up the policy, which is readily done in Tianshou.

In [11]:
policies = []
for net, optim in zip(nets, optims):
    policy = ts.policy.DQNPolicy(
        model=net, 
        optim=optim, 
        discount_factor=0.99,
        estimation_step=1,
        target_update_freq=50
    )
    policies.append(policies)

Then, we need to set up the collectors, i.e., the objects that will be interacting with the environment according to the above policy and collect the generated data.

In classical DQN fashion, we store the data in a replay buffer.

In [12]:
train_collectors = []
test_collectors = []
for policy in policies:
    train_collector = ts.data.Collector(policy, train_envs, ts.data.VectorReplayBuffer(1000, num_train_envs))
    train_collectors.append(train_collector)

    test_collector = ts.data.Collector(policy, test_envs)
    test_collectors.append(test_collector)

## Training

In [13]:
num_epochs = 15
num_steps_per_epoch = 2000
step_per_collect = 10
episode_per_test = 5
batch_size = 30 # one second of data (videos are at 30FPS)

timestamp = datetime.now().strftime("%d%m%Y-%H%M%S")
log_path = os.path.join("logs", "dqn", "base", f"video_{vid_filename}", f"ensemble_{n_subjects}", timestamp)
writer = SummaryWriter(log_path)
logger = TensorboardLogger(writer)

In [14]:
results = []
for policy, train_collector, test_collector in zip(policies, train_collectors, test_collectors):
    result = ts.trainer.offpolicy_trainer(
        policy, 
        train_collector, 
        test_collector,
        max_epoch=num_epochs,
        step_per_epoch=num_steps_per_epoch,
        step_per_collect=step_per_collect,
        episode_per_test=episode_per_test,
        batch_size=batch_size,
        logger=logger,
    )
    results.append(result)

Epoch #1: 3001it [00:06, 445.45it/s, env_step=3000, len=120, loss=1.504, n/ep=2, n/st=10, rew=56.35]                          


Epoch #1: test_reward: 153.505796 ± 126.639527, best_reward: 154.035783 ± 125.348117 in #0


Epoch #2: 3001it [00:06, 454.86it/s, env_step=6000, len=120, loss=1.710, n/ep=2, n/st=10, rew=47.47]                          


Epoch #2: test_reward: 153.505796 ± 124.430770, best_reward: 154.035783 ± 125.348117 in #0


Epoch #3: 3001it [00:06, 445.62it/s, env_step=9000, len=120, loss=2.095, n/ep=2, n/st=10, rew=49.39]                          


...

Epoch #18: test_reward: 153.505796 ± 125.293721, best_reward: 154.035783 ± 125.348117 in #0


Epoch #19: 3001it [00:06, 454.56it/s, env_step=57000, len=120, loss=4.958, n/ep=2, n/st=10, rew=55.25]                          


Epoch #19: test_reward: 153.505796 ± 115.376002, best_reward: 154.035783 ± 125.348117 in #0


Epoch #20: 3001it [00:06, 454.52it/s, env_step=60000, len=120, loss=2.159, n/ep=2, n/st=10, rew=53.47]                          


Epoch #20: test_reward: 153.505796 ± 127.634869, best_reward: 154.035783 ± 125.348117 in #0


In [15]:
results

{'duration': '138.74s',
 'train_time/model': '125.34s',
 'test_step': 37779,
 'test_episode': 105,
 'test_time': '5.06s',
 'test_speed': '7465.91 step/s',
 'best_reward': 154.03578341935645,
 'best_result': '154.04 ± 125.35',
 'train_step': 60000,
 'train_episode': 200,
 'train_time/collector': '8.34s',
 'train_speed': '448.83 step/s'}

In [16]:
for policy, train_envs in zip(policies, all_train_envs):
    policy.eval()
    policy.set_eps(0.05)

    collector = ts.data.Collector(policy, train_envs)
    print(collector.collect(n_episode=10))

{'n/ep': 10,
 'n/st': 3000,
 'rews': array([ 55.09495263,  50.14317183,  53.25710873,  58.55555754,
         59.01095125,  47.6988985 , 220.29657369,  52.25722118,
        368.95651349, 316.79558103]),
 'lens': array([120, 120, 120, 120, 120, 120, 510, 150, 810, 810]),
 'idxs': array([3, 4, 3, 4, 3, 4, 2, 4, 0, 1]),
 'rew': 128.20665298717904,
 'len': 300.0,
 'rew_std': 118.72347264807837,
 'len_std': 279.4995527724508}